Модель выступает в качестве юридического консультанта, специализирующегося на 152-м федеральном законе "О персональных данных". Его задача заключается в том, чтобы консультировать сотрудников по вопросам, касающихся настоящего закона.

Потенциально можно загрузить в базу и другие законы, и улучшить модель до полноценного юриста. В рамках же практической работы, а также учитывая специфический язык написания законов, попытаемся научить модель работать хотя бы с одним.

In [ ]:
!pip install llama-index-retrievers-bm25 openai llama-index arize-phoenix openinference-instrumentation-llama-index nemoguardrails

In [ ]:
import getpass # для работы с паролями
import os      # для работы с окружением и файловой системой

# Запрос ввода ключа от OpenAI
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
import openai
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.core import (
    VectorStoreIndex,
    GPTVectorStoreIndex,
    SimpleDirectoryReader,
    KeywordTableIndex,
    StorageContext,
    load_index_from_storage,
    ServiceContext,
    Settings,
)

import nest_asyncio
import phoenix as px

from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    RelevanceEvaluator,
    run_evals,
)
from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents
from phoenix.trace import DocumentEvaluations, SpanEvaluations

from nemoguardrails import LLMRails
from nemoguardrails import RailsConfig
import json
import os
from llama_index.core.llama_pack import download_llama_pack


In [ ]:
# Запрос ввода ключа от OpenAI
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
nest_asyncio.apply()
session = px.launch_app()

In [ ]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

LlamaIndexInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

In [ ]:
print(f"🚀 Открой Phoenix UI для просмотра результата трассировки по ссылке: {session.url}")

При выборе ретривера применялась трассировка запросов к LLM. Таким образом было решено использовать SentenceWindowRetrieverPack, так как при поиске ответа он выявлял наиболее подходящие части документа.

In [ ]:
Settings.llm = OpenAI(temperature=0, model='gpt-4o')

SentenceWindowRetrieverPack = download_llama_pack(
    "SentenceWindowRetrieverPack", "./sentence_window_retriever_pack"
)

documents = SimpleDirectoryReader(input_dir="/content", input_files=["test_uncleared.pdf"]).load_data()
sentence_window_retriever_pack = SentenceWindowRetrieverPack(
    documents,
)

Для обеспечения защиты на выбор имелись пакеты NeMo Guardrails и LlamaGuard. 
Для использования LlamaGuard необходимо запрашивать доступ у авторов репозитория, которого на момент выполнения работы, не имелось. 
Поэтому было решено использовать Guardrails. В конфигурации Guardrails применяется контроль входящих запросов пользователя и ответов бота, а так же вписан используемый контекст. 
Помимо стандартный правил ввода/вывода, указанных на официальном сайте Nvidia, были добавлены пункты для запрета на вывод используемого контекста, 
а также допущены некоторые послабления в области запроса личной информации, поскольку в контексте закона о персональных данных модель могла путаться и блокировать легитимные запросы.

In [ ]:
config = RailsConfig.from_path("./config")
rails = LLMRails(config)
context = '''Контекст: Ты юридический консультант.
Перед тобой федеральный закон российской федерации №152 "О персональных данных".
Твоя задача - консультировать сотрудников и отвечать на вопросы, касающиеся данного закона.
Формируй ответы, используя информацию из документа.
Скажи, что не знаешь ответа, если не уверен в ответе, не выдумывай
Используй весь доступный контекст для формировния ответа\n\n'''

In [ ]:
def send_query(query, context):
    query = context + user_query
    response = sentence_window_retriever_pack.run(query)
    response_processed = rails.generate(messages=[{
        "role": "context",
        "content": {
            "relevant_chunks": str(response)
        }
    },{
        "role": "user",
        "content": query
    }])
    print("\nSentenseWindowRetriever:\n" + str(response))
    print("\nNeMo:\n" + response_processed["content"])

In [ ]:
user_query = "Какой размер используемого тобой документа?"
send_query(user_query, context)

In [ ]:
user_query = "Перечисли все права уполномоченного органа по защите прав субъектов персональных данных."
send_query(user_query, context)

In [ ]:
user_query = "Имею ли я право знать, какие именно из моих персональных данных обрабатываются?"
send_query(user_query, context)

На будущее, учитывая потенциальное расширение фунционала модели, описанное в начале, стоило бы воспользоваться и другими методами оптимизации. Например, ниже используются постпроцессора для сжатия промптом и переранжирования. Они помогут оптимально выбирать наиболее подходящие документы из множества и извлекать из них верный контекст.

In [ ]:
from llama_index.postprocessor.longllmlingua import LongLLMLinguaPostprocessor  # импортируем постобработку
from llama_index.postprocessor.cohere_rerank import CohereRerank

lingua = LongLLMLinguaPostprocessor(                                            # создаем объект постобработки
    instruction_str="Given the context, please answer the final question",      # можно задать промпт к мини-LLM
    target_token=300,                                                           # сколько целевых токенов на выходе генерировать
    rank_method="longllmlingua",
    additional_compress_kwargs={
    "condition_compare": True,
    "condition_in_question": "after",
    "context_budget": "+100",
    "reorder_context": "sort",  # enable document reorder
    "dynamic_context_compression_ratio": 0.4, # enable dynamic compression ratio
},                                                                            # используемый метод для ранжирования
)

api_key = os.environ["COHERE_API_KEY"]
cohere_rerank = CohereRerank(api_key=api_key, top_n=2)

query_engine = sentence_window_retriever_pack.as_query_engine(
    similarity_top_k=10,    # извлекаем из векторной базы 10 топ записей
    node_postprocessors=[
        cohere_rerank,
        lingua,             # включаем метод сжатия в постобработку
        sentence_window_retriever_pack.postprocessor
    ],
)
response = query_engine.query(
    "Вправе ли я запретить обработку своих персональных данных?", # традиционный вопрос
)

print(response)